In [ ]:
!pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks

In [ ]:
api_key=""

In [ ]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec
index_name = "hybrid-searh-langchain-pinecone"

pc = Pinecone(api_key=api_key)

#create the index

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric='dotproduct',
        spec=ServerlessSpec(cloud='aws', region='us-east-1'),
    )

In [ ]:
index=pc.Index(index_name)
index

In [ ]:
#vector embedding and sparse matrix

import os
from dotenv import load_dotenv
load_dotenv()

os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")


from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

In [ ]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default()
bm25_encoder

In [ ]:
sentences = [
    "In 2021 I visited chennai",
    "In 2022 I visited Tirupati",
    "In 2023 I visited Pondicherry",
    ]

bm25_encoder.fit(sentences)

bm25_encoder.dump("bm25_values.json")

bm25_encoder = BM25Encoder().load("bm25_values.json")

In [ ]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings, sparse_encoder=bm25_encoder, index=index)

In [ ]:
retriever

In [ ]:
retriever.add_texts(
    [
    "In 2021 I visited chennai",
    "In 2022 I visited Tirupati",
    "In 2023 I visited Pondicherry",
    ]
)

In [ ]:
retriever.invoke("What city did i visit last")